In [ ]:
import pandas as pd
import warnings
import os
import plotly.graph_objects as go
from dotenv import load_dotenv
from google.cloud import bigquery

warnings.filterwarnings(
    "ignore", "Your application has authenticated using end user credentials"
)

load_dotenv()
LOCATION = "europe-north1"
PROJECT = os.environ["GCP_TEAM_PROJECT_ID"]

client = bigquery.Client(location=LOCATION, project=PROJECT)

norsk_dag={
    'Monday': 'Mandag',
    'Tuesday': 'Tirsdag',
    'Wednesday': 'Onsdag',
    'Thursday': 'Torsdag',
    'Friday': 'Fredag',
    'Saturday': 'Lørdag',
    'Sunday': 'Søndag',
    }

# 1. Mottatte søknader ny sykepengeløsning

Dette handler om sykepengesøknader som leveres fra sykepengesøknadsmottaket og til ny sykepengeløsning.
Per juli 2023 omfatter dette arbeidstaker-søknader, med unntak av kode 6 (og altså ikke frilansere, næringsdrivende etc).
Søknader som er korrigeringer på tidligere søknader telles, både korrigert og korrigerende.

Datoen som refereres til er datoen for når søknaden ble sendt første gang (så dersom en søknad oppdateres så er datoen fortsatt den originale innsendingsdatoen).


In [ ]:
import datetime as dt

print(f"Nøkkeltall per {dt.datetime.now().date()}")

## Nøkkeltall - inneværende måned


In [ ]:
QUERY = """--sql
    SELECT
        COUNTIF(DATE(soknad_mottatt, 'Europe/Oslo') = DATE_SUB(DATE(CURRENT_TIMESTAMP(), 'Europe/Oslo'), INTERVAL 1 DAY)) as `Mottatt i går`,
        COUNTIF(DATE(soknad_mottatt, 'Europe/Oslo') = DATE(CURRENT_TIMESTAMP(), 'Europe/Oslo')) as `Mottatt i dag`,
        COUNTIF(DATE(soknad_mottatt, 'Europe/Oslo') >= DATE_SUB(DATE(CURRENT_TIMESTAMP(), 'Europe/Oslo'), INTERVAL 1 WEEK) AND DATE(soknad_mottatt, 'Europe/Oslo') < DATE(CURRENT_TIMESTAMP(), 'Europe/Oslo')) as `Mottatt siste syv hele dager`,
        COUNTIF(DATE(soknad_mottatt, 'Europe/Oslo') >= DATE_TRUNC(DATE(CURRENT_TIMESTAMP(), 'Europe/Oslo'), MONTH)) as `Mottatt denne måneden`
    FROM `styringsinfo_dataset.styringsinfo_sendt_soknad_view`
"""

query_job = client.query(QUERY)
rows = query_job.result()

stats = [dict(row) for row in rows][0]

for key, value in stats.items():
    print(f'{key}: {value}')

In [ ]:
QUERY = f"""
    SELECT
        FORMAT_TIMESTAMP('%G%V', soknad_mottatt, 'Europe/Oslo') as week,
        COUNT(*) as total,
        COUNTIF(FORMAT_TIMESTAMP('%A', soknad_mottatt, 'Europe/Oslo') = 'Monday') as Monday,
        COUNTIF(FORMAT_TIMESTAMP('%A', soknad_mottatt, 'Europe/Oslo') = 'Tuesday') as Tuesday,
        COUNTIF(FORMAT_TIMESTAMP('%A', soknad_mottatt, 'Europe/Oslo') = 'Wednesday') as Wednesday,
        COUNTIF(FORMAT_TIMESTAMP('%A', soknad_mottatt, 'Europe/Oslo') = 'Thursday') as Thursday,
        COUNTIF(FORMAT_TIMESTAMP('%A', soknad_mottatt, 'Europe/Oslo') = 'Friday') as Friday,
        COUNTIF(FORMAT_TIMESTAMP('%A', soknad_mottatt, 'Europe/Oslo') = 'Saturday') as Saturday,
        COUNTIF(FORMAT_TIMESTAMP('%A', soknad_mottatt, 'Europe/Oslo') = 'Sunday') as Sunday
    FROM `styringsinfo_dataset.styringsinfo_sendt_soknad_view`
    GROUP BY week
    ORDER BY week DESC
"""

query_job = client.query(QUERY)

df_ukedager = query_job.to_dataframe()

# Usikker på hvordan jeg får til å rendre Markdown med Quarto
df_ukedager

In [ ]:
from plotly.subplots import make_subplots
import pandas_gbq

# SQL query to count number of rows per day and calculate 7 day moving average
QUERY = """
SELECT 
  date, 
  count,
  AVG(count) OVER (
    ORDER BY date
    ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
  ) AS moving_avg
FROM (
  SELECT 
    DATE(soknad_mottatt) AS date,
    COUNT(*) AS count
  FROM 
    `styringsinfo_dataset.styringsinfo_sendt_soknad_view`
  GROUP BY 
    DATE(soknad_mottatt)
)
WHERE date > '2023-03-01'
ORDER BY 
  date
"""

# Execute the query and load the data into a pandas DataFrame
df = pandas_gbq.read_gbq(QUERY, PROJECT, progress_bar_type="None")

# Create subplots, which will have a bar chart and a line chart
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bar chart for daily counts
fig.add_trace(
    go.Bar(x=df['date'], y=df['count'], name='Antall per dag'),
    secondary_y=False,
)

# Add line chart for 7 day moving average
fig.add_trace(
    go.Scatter(x=df['date'], y=df['moving_avg'], name='Snitt over 7 dager'),
    secondary_y=False,
)

# Set x and y axis labels
fig.update_layout(
    title_text="Antall mottatt per dag og 7-dagers glidende gjennomsnitt",
    xaxis_title="Dato",
    yaxis_title="Antall per dag",
    yaxis2_title="7-dagers glidende gjennomsnitt",
)

# Show the figure
fig.show()

In [ ]:
df = df_ukedager
# Create a new plotly figure
fig = go.Figure()

# Create a line for each weekday
for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
    fig.add_trace(go.Scatter(x=df['week'], y=df[day], mode='lines', name=norsk_dag[day]))

# Update the layout
fig.update_layout(
    title="Antall per ukedag for hver uke",
    xaxis_title="Uke",
    yaxis_title="Antall",
)

# Show the figure
fig.show()

In [ ]:
import datetime as dt

print(f'Sist oppdatert: {dt.datetime.now()}')